In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report
import torch
from torch.utils.data import Dataset, DataLoader
# from transformers import DebertaV3Tokenizer, DebertaV3ForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, AdamW, get_linear_schedule_with_warmup

from tqdm import tqdm
import re



# 添加数据文件路径到配置
CONFIG = {
    # 数据参数
    'data_file': '/mnt/cfs/huangzhiwei/BAE2025/data/mrbench_v3_devset.json',            # 数据文件路径
    
    # 模型参数
    'model_name': '/mnt/cfs/huangzhiwei/BAE2025/models/deberta-v3-base',  # 模型名称
    'max_length': 512,                          # 文本最大长度
    'dropout': 0.2,                             # dropout率
    
    # 训练参数
    'learning_rate': 2e-5,                      # 学习率
    'batch_size': 16,                           # 批次大小
    'epochs': 40,                               # 最大训练轮次
    'warmup_steps': 0,                          # 预热步数
    'weight_decay': 0.01,                       # 权重衰减
    'gradient_clip': 1.0,                       # 梯度裁剪
    
    # 早停参数
    'early_stopping': True,                     # 是否启用早停
    'patience': 6,                              # 容忍轮次
    'min_delta': 0.001,                         # 最小改进阈值
    'metric_for_early_stopping': 'macro_f1',    # 早停指标 ('macro_f1', 'weighted_f1', 'accuracy')
    
    # 交叉验证参数
    'n_splits': 5,                              # 交叉验证折数
    'random_state': 42,                         # 随机种子
    
    # 其他配置
    'save_model': False,                        # 是否保存最佳模型
    'model_path': '/mnt/cfs/huangzhiwei/BAE2025/projects/checkpoints_debertav3/best_model.pt',              # 模型保存路径
}



# 数据加载函数
def load_data(file_path):
    """
    从文件加载数据
    
    参数:
        file_path: 文件路径，可以是JSON文件或包含JSON的文本文件
        
    返回:
        数据列表
    """
    print(f"Loading data from {file_path}")
    
    # 检查文件扩展名
    if file_path.endswith('.json'):
        # 直接作为JSON文件读取
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                print(f"Successfully loaded JSON data with {len(data)} entries")
                return data
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON file: {e}")
            raise
    else:
        # 尝试从文本文件中提取JSON内容
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 尝试从文本中提取JSON数组
        try:
            # 查找方括号包围的内容
            json_pattern = re.compile(r'\[\s*\{.*?\}\s*(?:,\s*\.{3,}\s*)?\]', re.DOTALL)
            json_match = json_pattern.search(content)
            
            if json_match:
                # 替换省略号为空列表
                json_str = json_match.group().replace('......', '[]')
                data = json.loads(json_str)
                print(f"Successfully extracted JSON data from text file with {len(data)} entries")
                return data
            else:
                # 尝试解析整个文件作为JSON
                try:
                    data = json.loads(content)
                    print(f"Successfully parsed entire file as JSON with {len(data)} entries")
                    return data
                except json.JSONDecodeError:
                    raise ValueError("Could not extract JSON data from the input file")
        except Exception as e:
            print(f"Error processing file: {e}")
            raise# 早停类
class EarlyStopping:
    def __init__(self, patience=CONFIG['patience'], min_delta=CONFIG['min_delta'], metric=CONFIG['metric_for_early_stopping']):
        """
        初始化早停机制
        
        参数:
            patience: 在停止前等待的轮次数量
            min_delta: 被视为改进的最小变化量
            metric: 用于早停的指标 ('macro_f1', 'weighted_f1', 'accuracy')
        """
        self.patience = patience
        self.min_delta = min_delta
        self.metric = metric
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_state = None
        
    def __call__(self, model, current_metrics):
        """
        检查是否应该停止训练
        
        参数:
            model: 当前模型
            current_metrics: 当前评估指标字典 (包含 'accuracy', 'macro_f1', 'weighted_f1')
            
        返回:
            early_stop: 是否应该停止训练
        """
        score = current_metrics[self.metric]
        
        if self.best_score is None:
            self.best_score = score
            self.best_model_state = model.state_dict().copy()
            return False
        
        # 如果当前分数没有显著改进
        if score < self.best_score + self.min_delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_state = model.state_dict().copy()
            self.counter = 0
            
        return self.early_stop


# Load the data
data = load_data(CONFIG['data_file'])

# Prepare dataset
texts = []
labels = []

label_map = {"Yes": 2, "To some extent": 1, "No": 0}
rev_label_map = {2: "Yes", 1: "To some extent", 0: "No"}

for item in data:
    conversation_history = item["conversation_history"]
    
    # Process each tutor response
    for tutor, response_data in item["tutor_responses"].items():
        if "annotation" in response_data and "Actionability" in response_data["annotation"]:
            # Include the tutor's response as the input text
            texts.append(response_data["response"])
            
            # Map the guidance label to a numeric value
            guidance_label = response_data["annotation"]["Actionability"]
            labels.append(label_map[guidance_label])

# Convert to pandas DataFrame for easier manipulation
df = pd.DataFrame({
    "text": texts,
    "label": labels
})

print(f"Total samples: {len(df)}")
print(f"Label distribution: {df['label'].value_counts().to_dict()}")

# Create a PyTorch dataset
class GuidanceDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=CONFIG['max_length']):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Training function
def train_model(model, train_dataloader, val_dataloader, device, epochs=CONFIG['epochs']):
    
    # # 首先冻结所有层
    # for param in model.parameters():
    #     param.requires_grad = False

    # # 然后只解冻最后几层
    # # 解冻分类器
    # for param in model.classifier.parameters():
    #     param.requires_grad = True

    # # 解冻最后N个encoder层（例如最后3层）
    # for i in range(len(model.deberta.encoder.layer) - 4, len(model.deberta.encoder.layer)):
    #     for param in model.deberta.encoder.layer[i].parameters():
    #         param.requires_grad = True
    
    optimizer = AdamW(model.parameters(), 
                     lr=CONFIG['learning_rate'], 
                     weight_decay=CONFIG['weight_decay'])
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=CONFIG['warmup_steps'], 
        num_training_steps=total_steps
    )
    
    # 初始化最佳指标和模型状态
    best_val_metric = 0
    best_model_state = None
    
    # 初始化早停机制
    early_stopping = None
    if CONFIG['early_stopping']:
        early_stopping = EarlyStopping(
            patience=CONFIG['patience'],
            min_delta=CONFIG['min_delta'],
            metric=CONFIG['metric_for_early_stopping']
        )
    
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        
        # Training
        model.train()
        train_loss = 0
        train_preds = []
        train_true = []
        
        for batch in tqdm(train_dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            model.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            train_loss += loss.item()
            
            # Get predictions for metrics
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            train_preds.extend(preds)
            train_true.extend(labels.cpu().numpy())
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG['gradient_clip'])
            optimizer.step()
            scheduler.step()
        
        # Calculate training metrics
        avg_train_loss = train_loss / len(train_dataloader)
        train_acc = accuracy_score(train_true, train_preds)
        train_report = classification_report(train_true, train_preds, target_names=[rev_label_map[i] for i in range(3)], output_dict=True)
        train_f1_weighted = train_report['weighted avg']['f1-score']
        train_f1_macro = train_report['macro avg']['f1-score']
        
        print(f"Training loss: {avg_train_loss:.4f}")
        print(f"Training accuracy: {train_acc:.4f}")
        print(f"Training weighted F1 score: {train_f1_weighted:.4f}")
        print(f"Training macro F1 score: {train_f1_macro:.4f}")
        
        # Validation
        model.eval()
        val_preds = []
        val_true = []
        
        with torch.no_grad():
            for batch in tqdm(val_dataloader):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                
                val_preds.extend(preds)
                val_true.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_true, val_preds)
        val_report = classification_report(val_true, val_preds, target_names=[rev_label_map[i] for i in range(3)], output_dict=True)
        val_f1_weighted = val_report['weighted avg']['f1-score']
        val_f1_macro = val_report['macro avg']['f1-score']
        
        print(f"Validation accuracy: {val_acc:.4f}")
        print(f"Validation weighted F1 score: {val_f1_weighted:.4f}")
        print(f"Validation macro F1 score: {val_f1_macro:.4f}")
        print(classification_report(val_true, val_preds, target_names=[rev_label_map[i] for i in range(3)]))
        
        # 跟踪当前指标，用于早停和保存最佳模型
        current_metrics = {
            'accuracy': val_acc,
            'weighted_f1': val_f1_weighted,
            'macro_f1': val_f1_macro
        }
        
        # 跟踪最佳模型（基于与早停相同的指标）
        current_metric = current_metrics[CONFIG['metric_for_early_stopping']]
        if current_metric > best_val_metric:
            best_val_metric = current_metric
            best_model_state = model.state_dict().copy()
            
            # 如果配置了保存模型，则保存模型
            if CONFIG['save_model']:
                torch.save(model.state_dict(), CONFIG['model_path'])
                print(f"Model saved to {CONFIG['model_path']} (best {CONFIG['metric_for_early_stopping']}: {best_val_metric:.4f})")
        
        # 早停检查
        if CONFIG['early_stopping'] and early_stopping(model, current_metrics):
            print(f"Early stopping triggered after epoch {epoch + 1}")
            # 恢复最佳模型状态
            model.load_state_dict(early_stopping.best_model_state)
            break
    
    # 如果训练完成但没有触发早停，确保加载最佳模型
    if best_model_state is not None and (not CONFIG['early_stopping'] or not early_stopping.early_stop):
        model.load_state_dict(best_model_state)
        print(f"Training completed. Loaded best model with {CONFIG['metric_for_early_stopping']}: {best_val_metric:.4f}")
    
    return model

# 5-fold cross-validation
def perform_cross_validation(df, n_splits=CONFIG['n_splits'], batch_size=CONFIG['batch_size'], epochs=CONFIG['epochs']):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=CONFIG['random_state'])
    fold_results = []
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load the tokenizer
    # tokenizer = DebertaV3Tokenizer.from_pretrained(CONFIG['model_name'])
    tokenizer = DebertaV2Tokenizer.from_pretrained(CONFIG['model_name'])
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
        print(f"Fold {fold + 1}/{n_splits}")
        
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]
        
        # Create datasets
        train_dataset = GuidanceDataset(
            train_df['text'].tolist(),
            train_df['label'].tolist(),
            tokenizer
        )
        
        val_dataset = GuidanceDataset(
            val_df['text'].tolist(),
            val_df['label'].tolist(),
            tokenizer
        )
        
        # Create dataloaders
        train_dataloader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True
        )
        
        val_dataloader = DataLoader(
            val_dataset,
            batch_size=batch_size
        )
        
        # Initialize model
        # model = DebertaV3ForSequenceClassification.from_pretrained(
        #     CONFIG['model_name'],
        #     num_labels=3,
        #     hidden_dropout_prob=CONFIG['dropout'],
        #     attention_probs_dropout_prob=CONFIG['dropout']
        # ).to(device)
        
        model = DebertaV2ForSequenceClassification.from_pretrained(
            CONFIG['model_name'],
            num_labels=3,
            hidden_dropout_prob=CONFIG['dropout'],
            attention_probs_dropout_prob=CONFIG['dropout']
        ).to(device)
        
        # Train and evaluate
        model = train_model(
            model,
            train_dataloader,
            val_dataloader,
            device,
            epochs=epochs
        )
        
        # Final evaluation on validation set
        model.eval()
        val_preds = []
        val_true = []
        
        with torch.no_grad():
            for batch in DataLoader(val_dataset, batch_size=batch_size):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                
                val_preds.extend(preds)
                val_true.extend(labels.cpu().numpy())
        
        fold_acc = accuracy_score(val_true, val_preds)
        fold_report = classification_report(
            val_true, 
            val_preds, 
            target_names=[rev_label_map[i] for i in range(3)],
            output_dict=True
        )
        
        fold_f1_weighted = fold_report['weighted avg']['f1-score']
        fold_f1_macro = fold_report['macro avg']['f1-score']
        
        fold_results.append({
            'fold': fold + 1,
            'accuracy': fold_acc,
            'f1_weighted': fold_f1_weighted,
            'f1_macro': fold_f1_macro,
            'report': fold_report
        })
        
        print(f"Fold {fold + 1} accuracy: {fold_acc:.4f}")
        print(f"Fold {fold + 1} weighted F1 score: {fold_f1_weighted:.4f}")
        print(f"Fold {fold + 1} macro F1 score: {fold_f1_macro:.4f}")
    
    # Calculate average metrics across folds
    avg_acc = np.mean([result['accuracy'] for result in fold_results])
    avg_f1_weighted = np.mean([result['f1_weighted'] for result in fold_results])
    avg_f1_macro = np.mean([result['f1_macro'] for result in fold_results])
    
    print(f"Average accuracy across {n_splits} folds: {avg_acc:.4f}")
    print(f"Average weighted F1 score across {n_splits} folds: {avg_f1_weighted:.4f}")
    print(f"Average macro F1 score across {n_splits} folds: {avg_f1_macro:.4f}")
    
    # Calculate average per-class metrics
    class_metrics = {label: {'precision': [], 'recall': [], 'f1-score': []} for label in rev_label_map.values()}
    
    for result in fold_results:
        for label in rev_label_map.values():
            if label in result['report']:
                class_metrics[label]['precision'].append(result['report'][label]['precision'])
                class_metrics[label]['recall'].append(result['report'][label]['recall'])
                class_metrics[label]['f1-score'].append(result['report'][label]['f1-score'])
    
    print("\nAverage per-class metrics across all folds:")
    for label, metrics in class_metrics.items():
        print(f"{label}:")
        for metric_name, values in metrics.items():
            avg_value = np.mean(values) if values else 0
            print(f"  {metric_name}: {avg_value:.4f}")
    
    return fold_results

# Save processed data to CSV (optional)
df.to_csv('/mnt/cfs/huangzhiwei/BAE2025/projects/checkpoints_debertav3/guidance_classification_data.csv', index=False)

# Execute cross-validation with the specified parameters
print("Starting cross-validation with the following parameters:")
for param, value in CONFIG.items():
    print(f"  {param}: {value}")

fold_results = perform_cross_validation(
    df, 
    n_splits=CONFIG['n_splits'],
    batch_size=CONFIG['batch_size'],
    epochs=CONFIG['epochs']
)

# Save the fold results (optional)
with open('/mnt/cfs/huangzhiwei/BAE2025/projects/checkpoints_debertav3/fold_results.json', 'w') as f:
    json.dump(fold_results, f, indent=2)

print("Classification task completed successfully.")

/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading data from /mnt/cfs/huangzhiwei/BAE2025/data/mrbench_v3_devset.json
Successfully loaded JSON data with 300 entries
Total samples: 2476
Label distribution: {2: 1310, 0: 797, 1: 369}
Starting cross-validation with the following parameters:
  data_file: /mnt/cfs/huangzhiwei/BAE2025/data/mrbench_v3_devset.json
  model_name: /mnt/cfs/huangzhiwei/BAE2025/models/deberta-v3-base
  max_length: 512
  dropout: 0.2
  learning_rate: 2e-05
  batch_size: 16
  epochs: 40
  warmup_steps: 0
  weight_decay: 0.01
  gradient_clip: 1.0
  early_stopping: True
  patience: 6
  min_delta: 0.001
  metric_for_early_stopping: macro_f1
  n_splits: 5
  random_state: 42
  save_model: False
  model_path: /mnt/cfs/huangzhiwei/BAE2025/projects/checkpoints_debertav3/best_model.pt
Using device: cuda


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /mnt/cfs/huangzhiwei/BAE2025/models/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fold 1/5


/mnt/cfs/anaconda3/envs/pykt-hzw3.8/lib/python3.8/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Validation accuracy: 0.7131
Validation weighted F1 score: 0.6909
Validation macro F1 score: 0.5772
                precision    recall  f1-score   support

            No       0.63      0.89      0.74       145
To some extent       0.37      0.14      0.21        70
           Yes       0.81      0.76      0.79       280

      accuracy                           0.71       495
     macro avg       0.60      0.60      0.58       495
  weighted avg       0.70      0.71      0.69       495

Epoch 7/40


100%|██████████| 124/124 [00:43<00:00,  2.87it/s]


Training loss: 0.3771
Training accuracy: 0.8693
Training weighted F1 score: 0.8647
Training macro F1 score: 0.8110


 10%|▉         | 3/31 [00:00<00:04,  5.83it/s]


KeyboardInterrupt: 